In [1]:
import requests
import pandas as pd
from datetime import datetime
import pymysql
import numpy as np
import time
import traceback2
from pycoingecko import CoinGeckoAPI

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
def get_todays_date():
    today = datetime.now()
    today = datetime(today.year, today.month, today.day)
    return today

In [4]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [5]:
def get_symbol_and_name_mappings():
    sql = "select * from notebook_all_required_symbols_cgk;"
    connection = __connect__()
    try:
        df = pd.read_sql(sql, connection)
        return df
    except Exception:
        traceback2.format_exc()
        return pd.DataFrame
    finally:
        connection.close()
    return

def get_coingecko_projects():
    data = cg.get_coins_list()
    df = pd.DataFrame(data)
    df['symbol'] = df['symbol'].str.upper()
    return df

def get_unique_saved_ids():
    sql = "select distinct id from cgk_coin_metainfo_bckup"
    df = pd.read_sql(sql, __connect__())
    return df["id"].unique().tolist()

def not_in_items(l1, l2):
    l = []
    for i in l1:
        if i not in l2:
            l.append(i)
    return l

def get_ids_to_pull():
    projects_df = get_coingecko_projects()
    symbol_name_mapps_df = get_symbol_and_name_mappings()
    saved_ids = get_unique_saved_ids()
    filtered_ids = projects_df[(projects_df['symbol'].isin(symbol_name_mapps_df['symbol'].unique().tolist()))
               &
               (projects_df['name'].isin(symbol_name_mapps_df['name_in_coingecko'].unique().tolist()))
               ].drop_duplicates().reset_index(drop=True)
    new_ids = not_in_items(filtered_ids['id'].unique().tolist(), saved_ids)
    print(new_ids)
    return filtered_ids[filtered_ids['id'].isin(new_ids)].reset_index(drop=True)

In [6]:
filtered_ids = get_ids_to_pull()
filtered_ids


['booster', 'bunicorn', 'cherryswap', 'coin', 'fleta', 'fruits', 'mimo-parallel-governance-token', 'pilot']


,id,symbol,name
0,booster,BOO,Booster
1,bunicorn,BUNI,Bunicorn
2,cherryswap,CHE,CherrySwap
3,coin,COIN,Coin
4,fleta,FLETA,FLETA
5,fruits,FRTS,Fruits
6,mimo-parallel-governance-token,MIMO,MIMO Parallel Governance Token
7,pilot,PTD,Pilot


### Meta Info

In [7]:
def insert_coingecko_coininfo(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(row["date"], row["id"], row["symbol"], row["name"], row["name"], row["categories"], 1)
        try:
            sql = """INSERT INTO `cgk_coin_metainfo` (`timestamp_dt`, `id`, `symbol`, `name`, `current_name`, `tags`, `is_active`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)
                """
            cursor.execute(sql, (dt, row["id"], row["symbol"], row["name"], row["name"], row["categories"], 1))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()


def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        ids.append(data['id'])
    return ids


def get_top_n_coins_market_data(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
    rows = rows[0:n]
    return rows


def get_coin_data(id, wait=40):
    try:
        coin_data = cg.get_coin_by_id(id=id, localization="false", market_data='true', tickers='false', sparkline='false')
        return coin_data if coin_data is not None else []
    except HTTPError as e:
        print(e)
        time.sleep(wait)
        get_coin_data(id, wait * 2)


def get_coin_info(top_n_ids):
    count = 0
    rows = []
    for id in top_n_ids:
        count = count + 1
        coin_data = get_coin_data(id)
        print(f"{count}. {id}")
        row = [coin_data['id'], coin_data['symbol'].upper(), coin_data['name'], ",".join(coin_data['categories']) if coin_data['categories'] else '']
        rows.append(row)
        time.sleep(.7)
    df = pd.DataFrame(rows, columns=['id', 'symbol', 'name', 'categories'])
    df['date'] = get_todays_date()
    return df

In [8]:
ids = filtered_ids['id'].unique().tolist()

In [9]:
meta_info = get_coin_info(ids)

1. booster
2. bunicorn
3. cherryswap
4. coin
5. fleta
6. fruits
7. mimo-parallel-governance-token
8. pilot


In [10]:
meta_info

,id,symbol,name,categories,date
0,booster,BOO,Booster,,2021-08-08
1,bunicorn,BUNI,Bunicorn,,2021-08-08
2,cherryswap,CHE,CherrySwap,,2021-08-08
3,coin,COIN,Coin,Decentralized Finance (DeFi),2021-08-08
4,fleta,FLETA,FLETA,Cryptocurrency,2021-08-08
5,fruits,FRTS,Fruits,,2021-08-08
6,mimo-parallel-governance-token,MIMO,MIMO Parallel Governance Token,,2021-08-08
7,pilot,PTD,Pilot,HECO Chain Ecosystem,2021-08-08


In [11]:
insert_coingecko_coininfo(meta_info)

2021-08-08 00:00:00 booster BOO Booster Booster  1
(1062, "Duplicate entry 'booster-BOO-Booster-Booster' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 bunicorn BUNI Bunicorn Bunicorn  1
(1062, "Duplicate entry 'bunicorn-BUNI-Bunicorn-Bunicorn' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 cherryswap CHE CherrySwap CherrySwap  1
(1062, "Duplicate entry 'cherryswap-CHE-CherrySwap-CherrySwap' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 coin COIN Coin Coin Decentralized Finance (DeFi) 1
(1062, "Duplicate entry 'coin-COIN-Coin-Coin' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 fleta FLETA FLETA FLETA Cryptocurrency 1
(1062, "Duplicate entry 'fleta-FLETA-FLETA-FLETA' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 fruits FRTS Fruits Fruits  1
(1062, "Duplicate entry 'fruits-FRTS-Fruits-Fruits' for key 'cgk_coin_metainfo.PRIMARY'")
2021-08-08 00:00:00 mimo-parallel-governance-token MIMO MIMO Parallel Governance Token MIMO Parallel G

### Hostorical Data And market Data

In [12]:

def insert_coingecko_data(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl'])
        try:
            sql = """INSERT INTO `coingecko_coin_mcap_supply_data` (`timestamp_dt`, `symbol`, `name`, `price`, `mrk_cap`,`max_supply`, `circulating_supply`, `total_supply`, `volume`, `tvl`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                `mrk_cap` = VALUES(mrk_cap),
                `price` = VALUES(price),
                `max_supply` = VALUES(max_supply),
                `circulating_supply` = VALUES(circulating_supply),
                `total_supply` = VALUES(total_supply),
                `volume` = VALUES(volume),
                `tvl` = VALUES(tvl)
                """
            cursor.execute(sql, (dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl']))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()
    return


def insert_coingecko_hist_data(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(row["date"], row["id"], row["symbol"], row["name"], row["price"], row["mrk_cap"], row['volume'])
        try:
            sql = """INSERT INTO `coingecko_coin_hist_data` (`timestamp_dt`, `id`, `symbol`, `name`, `price`, `mrk_cap`,`volume`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                `mrk_cap` = VALUES(mrk_cap),
                `price` = VALUES(price),
                `volume` = VALUES(volume)
                """
            cursor.execute(sql, (dt, row["id"], row["symbol"], row["name"], row["price"], row["mrk_cap"], row['volume']))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()
    return


def get_todays_date():
    today = datetime.now()
    today = datetime(today.year, today.month, today.day)
    return today


def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        ids.append(data['id'])
    return ids


def get_top_n_coins_market_data(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
    rows = rows[0:n]
    return rows


def get_coin_data(id):
    try:
        coin_data = cg.get_coin_by_id(id=id, localization="false", market_data='true', tickers='false', sparkline='false')
        return get_coin_market_data(coin_data)
    except Exception:
        print(traceback2.format_exc())
        return []


def get_data(market_data, key):
    if len(key) == 2:
        return market_data[key[0]]['usd'] if key[0] in market_data and market_data[key[0]] else np.nan
    elif len(key) == 1:
        return market_data[key[0]] if key[0] in market_data else np.nan
    else:
        return np.nan


def get_coin_market_data(coin):
    market_data = coin['market_data']
    return [
        get_data(coin, ['symbol']).upper(),
        get_data(coin, ['name']),
        get_data(market_data, ['current_price', 'usd']),
        get_data(market_data, ['market_cap', 'usd']),
        get_data(market_data, ['total_volume', 'usd']),
        get_data(market_data, ['circulating_supply']),
        get_data(market_data, ['total_supply']),
        get_data(market_data, ['max_supply']),
        get_data(market_data, ['total_value_locked', 'usd']),
        # get_data(market_data, ['fully_diluted_valuation', 'usd']),
        # get_data(market_data, ['roi']),
    ]


def get_coin_historical_data(id, days=90):
    data = cg.get_coin_market_chart_by_id(id=id, vs_currency="usd", days=days, interval="hourly")
    mcap_df = pd.DataFrame(data['market_caps'], columns=['date', 'mrk_cap'])
    price_df = pd.DataFrame(data['prices'], columns=['date', 'price'])
    volume_df = pd.DataFrame(data['total_volumes'], columns=['date', 'volume'])
    merged_df = pd.merge(mcap_df, price_df, left_on=['date'], right_on=['date'])
    merged_df = pd.merge(merged_df, volume_df, left_on=['date'], right_on=['date'])
    merged_df['date'] = pd.to_datetime(pd.to_datetime(merged_df['date'], unit='ms').dt.strftime('%Y-%m-%d %H:00:00'))
    merged_df['id'] = id
    merged_df
    return merged_df


def get_coin_historical_chart_data(id, name, symbol):
    df = get_coin_historical_data(id, days=90)
    df['name'] = name
    df['symbol'] = symbol
    return df


def join_symbols(old_list, new_list):
    new_symbols = old_list.copy()
    for symbol in new_list:
        if symbol not in new_symbols:
            new_symbols.append(symbol)
    return new_symbols


def pull_top_n_coins_market_data(top_n_ids):
    historical_data_dfs = pd.DataFrame(columns=['date', 'mcap', 'volume', 'id', 'name', 'symbol'])
    rows = []
    time.sleep(1)
    index = 0
    batch = 3
    while index < len(top_n_ids):
        ids = top_n_ids[index: index + batch]
        for id in ids:
            print(f"{index}. Pulling market data: {id}")
            time.sleep(1)
            row = get_coin_data(id)
            if row and len(row):
                hist_df = get_coin_historical_chart_data(id, row[1], row[0])
                historical_data_dfs = historical_data_dfs.append(hist_df)
            rows.append(row)
        index = index + batch
        time.sleep(1)
    df = pd.DataFrame(rows, columns=['symbol', 'name', 'price', 'mrk_cap', 'volume', 'circulating_supply', 'total_supply', 'max_supply', 'tvl'])
    df['date'] = get_todays_date()
    return [df, historical_data_dfs]

In [13]:
data = pull_top_n_coins_market_data(ids)

0. Pulling market data: booster
0. Pulling market data: bunicorn
0. Pulling market data: cherryswap
3. Pulling market data: coin
3. Pulling market data: fleta


KeyboardInterrupt: 

In [ ]:
insert_coingecko_data(data[0])
insert_coingecko_hist_data(data[1])

In [ ]:
len(data[1]['name'].unique().tolist())